# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[HSE][ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier, Pool
from collections import Counter

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [2]:
def gini(x):
    result = 0
    for y in np.unique(x):
        p_y = (x==y).sum() / len(x)
        result += p_y * (1 - p_y)
    return result
    
def entropy(x):
    result = 0
    for y in np.unique(x):
        p_y = (x==y).sum() / len(x)
        result += p_y * np.log(p_y)
    return -result

def gain(left_y, right_y, criterion):
    node_size = len(left_y) + len(right_y)
    return criterion(np.hstack((left_y, right_y))) - len(left_y) / node_size * criterion(left_y) - len(right_y) / node_size * criterion(right_y)

In [3]:
class DecisionTreeLeaf:
    def __init__(self, y):
        self.y = y

class DecisionTreeNode:
    def __init__(self, split_dim, left, right):
        self.split_dim = split_dim
        self.left = left
        self.right = right

In [4]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.criterion = gini if criterion == 'gini' else entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = int(np.sqrt(X.shape[1])) if max_features == 'auto' else max_features
        self.unique = np.unique(X)
        self.root = self.build(X, y, 0)
        
    def build(self, X, y, cur_depth):

        if cur_depth == self.max_depth:
            return DecisionTreeLeaf(max(Counter(y), key=counts.get))
        
        max_gain = -1
    
        rand_features = np.random.choice(np.arange(X.shape[1]), size=self.max_features, replace=False)
    
        for i in rand_features:
            y_left = y[X[:,i] == self.unique[0]]
            y_right = y[X[:,i] == self.unique[1]]
            cur_gain = gain(y_left, y_right, self.criterion)
            if cur_gain > max_gain:
                max_gain = cur_gain
                split_dim = i        
        
        left_mask = X[:,split_dim] == self.unique[0]
        right_mask = X[:,split_dim] == self.unique[1]
        
        if left_mask.sum() <= self.min_samples_leaf or right_mask.sum() <= self.min_samples_leaf:
            return DecisionTreeLeaf(max(Counter(y), key=counts.get))   
        
        left_child = self.build(X[left_mask], y[left_mask], cur_depth+1)
        right_child = self.build(X[right_mask], y[right_mask], cur_depth+1)

        return DecisionTreeNode(split_dim, left_child, right_child)    
    
    def predict(self, X):
        
        result = []
        
        for x in X:
            cur_node = self.root
            while type(cur_node) != DecisionTreeLeaf:
                if x[cur_node.split_dim]==self.unique[0]:
                    cur_node = cur_node.left
                else:
                    cur_node = cur_node.right
            
            result.append(cur_node.y)
        
        return np.array(result)

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [5]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = gini if criterion == 'gini' else entropy
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators
        
        self.trees = []
        self.oob = []
    
    def fit(self, X, y):
        for i in range(self.n_estimators):
            random_ind = np.random.choice(np.arange(len(X)), size=len(X))
            
            random_ind_set = set(random_ind)
            all_ind_set = set(np.arange(len(X)))
            oob = np.array(list(all_ind_set - random_ind_set))
            self.oob.append(oob)
             
            X_rand = X[random_ind]
            y_rand = y[random_ind]
            tree = DecisionTree(X_rand, y_rand, self.criterion, self.max_depth, self.min_samples_leaf, self.max_features)
            self.trees.append(tree)
            
        self.calculate_feature_importances(X,y)
        
    def calculate_feature_importances(self, X, y):
        importances = []
        for i in range(self.n_estimators):
            X_oob = X[self.oob[i]]
            y_oob = y[self.oob[i]]
            
            err_oob = (self.trees[i].predict(X_oob) != y_oob).mean()

            feature_importance = []

            for feature in range(X.shape[1]):
                X_shuffled = np.copy(X_oob)
                np.random.shuffle(X_shuffled[:,feature])

                err_oob_f = (self.trees[i].predict(X_shuffled) != y_oob).mean()
                feature_importance.append(err_oob_f - err_oob)
            
            importances.append(feature_importance)
        
        self.importances = np.array(importances).mean(axis=0)
            
    
    def predict(self, X):
        results_by_trees = np.array([tree.predict(X) for tree in self.trees]).T
        
        result = []
        for x_result in results_by_trees:
            c = Counter(x_result)
            result.append(max(c, key=c.get))
        
        return result

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [6]:
def feature_importance(rfc):
    return rfc.importances

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [7]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

Accuracy: 1.0
Importance: [9.85964545e-04 1.61264244e-04 1.58597708e-01 1.61560849e-01
 3.36364359e-01 1.13334564e-03]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [8]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [9]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [10]:
rfc = RandomForestClassifier(n_estimators=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.7036569987389659
Most important features:
1. ovsyanochan
2. mudakoff
3. styd.pozor
4. 4ch
5. rhymes
6. dayvinchik
7. rapnewrap
8. iwantyou
9. tumblr_vacuum
10. pixel_stickers
11. pravdashowtop
12. leprum
13. bot_maxim
14. bestad
15. reflexia_our_feelings
16. bog_memes
17. xfilm
18. ultrapir
19. top_screens
20. soverwenstvo.decora


#### Пол

In [11]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

Accuracy: 0.8385876418663304
Most important features:
1. 40kg
2. modnailru
3. girlmeme
4. mudakoff
5. zerofat
6. i_d_t
7. femalemem
8. be.beauty
9. 9o_6o_9o
10. beauty
11. reflexia_our_feelings
12. thesmolny
13. sh.cook
14. woman.blog
15. bon
16. rapnewrap
17. recipes40kg
18. igm
19. cook_good
20. h.made


### CatBoost
В качестве альтернативы попробуем CatBoost. 

Установить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [12]:
X, y = synthetic_dataset(1000)

model = CatBoostClassifier(n_estimators=100, loss_function='MultiClass', silent=True)
model.fit(X,y)

print("Accuracy:", np.mean(model.predict(X).reshape(-1) == y))
print("Importance:", model.get_feature_importance())

Accuracy: 1.0
Importance: [1.33622563e-03 4.73973411e-03 2.79186827e+01 2.78502633e+01
 4.42238354e+01 1.14261656e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [13]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [14]:
model = CatBoostClassifier(n_estimators=10, loss_function='MultiClass', silent=True)
model.fit(X_train, y_age_train)

print("Accuracy:", np.mean(model.predict(X_test).reshape(-1) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.7011349306431274
Most important features:
1. ovsyanochan
2. styd.pozor
3. rhymes
4. tumblr_vacuum
5. 4ch
6. dayvinchik
7. pixel_stickers
8. fuck_humor
9. leprum
10. xfilm


#### Пол

In [15]:
model = CatBoostClassifier(n_estimators=10, loss_function='MultiClass', silent=True)
model.fit(X_train, y_sex_train)

print("Accuracy:", np.mean(model.predict(X_test).reshape(-1) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(model.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.8272383354350568
Most important features:
1. 40kg
2. mudakoff
3. girlmeme
4. zerofat
5. cook_good
6. thesmolny
7. igm
8. modnailru
9. bon
10. 9o_6o_9o
